In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)
from spyne.application import Application
from spyne.decorator import srpc
from spyne.service import ServiceBase
from spyne.model.primitive import Integer
from spyne.model.primitive import Unicode
from spyne.model.complex import Iterable
from spyne.protocol.http import HttpRpc
from spyne.protocol.json import JsonDocument
from spyne.server.wsgi import WsgiApplication
import threading
import hashlib
import redis

class DiminishedTriadService(ServiceBase):

    def __init__(self):
        print "I am being instantiated !!!!"
        
    @srpc(Unicode, _returns=Unicode)
    def get(key):
        backs = getNextNBackends(key)
        #r = redis.StrictRedis(host='localhost', port=30001, db=0)
        print "lenght of backs : ", len(backs)
        print "backs : ", backs
        print "global map in get():", hash_to_back_map
        for back in backs:
            addr = back.split(":")[0]
            port = back.split(":")[1]
            try:
                r = redis.StrictRedis(host=addr, port=port, db=0)
                return r.get(key)
            except:
                print "failed to get key: ", key ," from addr: ",addr," port: ", port
                continue
            
    @srpc(Unicode,Unicode, _returns=Unicode)
    def set(key,value):
        #r = redis.StrictRedis(host='localhost', port=30001, db=0)
        backs = getNextNBackends(key)
        print "lenght of backs : ", len(backs)
        print "backs : ", backs
        print "global map in set():", hash_to_back_map
        for back in backs:
            addr = back.split(":")[0]
            port = back.split(":")[1]
            try:
                r = redis.StrictRedis(host=addr, port=port, db=0)
                r.set(key,value)
            except:
                print "failed to set key: ", key ," to addr: ",addr," port: ", port
                continue
        return OK
    
    @srpc(Unicode, _returns=Unicode)
    def update_hash_to_back_map(addrs):
        try:
            lock.acquire()
            new_hash_to_back_map = {}
            for addr in addrs.split(","):
                new_hash_to_back_map[hash(addr)] = addr
            hash_to_back_map.clear()
            for k in new_hash_to_back_map:
                hash_to_back_map[k] = new_hash_to_back_map[k]
        except:
            e = sys.exc_info()[0]
            print e
            return e
        finally:
            lock.release()
            return OK
    
    
    @srpc(_returns=Unicode)
    def getAOFLog():
        f = open(AOFPATH,'r')
        string = f.read()
        print "getAOF returning ", string
        return string
    
    @srpc(Unicode,_returns=Integer)
    def getStrlen(key):
        backs = getNextNBackends(key)
        for back in backs:
            try:
                addr = back.split(":")[0]
                port = back.split(":")[1]
                r = redis.StrictRedis(host=addr, port=port, db=0)
                return r.get(key)
            except:
                continue
                
    @srpc(Unicode,Unicode, _returns=Integer)
    def append(key,append_str):
        backs = getNextNBackends(key)
        for back in backs:
            try:
                addr = back.split(":")[0]
                port = back.split(":")[1]
                r = redis.StrictRedis(host=addr, port=port, db=0)
                return r.append(key,append_str)
            except:
                print "failed to append to key: ", key ," append_str : ", append_str ," to addr: ", addr , "port: " , port
                continue
        
         
        
    
    
############################### End of Class DiminishedTriadService ########################################

NUMBACKENDS = 100 # this determines what we are modding by
NUMBACKUPS = 1
OK = "ok"

def hash(key):
    return int(hashlib.sha1(key).hexdigest(), 16)

lock = threading.RLock()
AOFPATH = "create-multiredis/appendonly-30001.aof"
hash_to_back_map = {
                            hash("127.0.0.1:30001"):"127.0.0.1:30001",
                            hash("127.0.0.1:30002"):"127.0.0.1:30002",
                            hash("127.0.0.1:30003"):"127.0.0.1:30003"
                    }


def getNextNBackends(key):
    try:
        lock.acquire()
        keys = hash_to_back_map.keys()
        print "keys : ", keys
        keys.append(hash(key))
        keys = sorted(keys)
        index = keys.index(hash(key))
        out = []
        for i in range(1,NUMBACKUPS+2):
            if keys[(index + i)%len(keys)] in hash_to_back_map and hash_to_back_map[keys[(index + i)%len(keys)]] not in out:
                out.append(hash_to_back_map[keys[(index + i)%len(keys)]])
        print "getNextNBackends for key : ",key," returning : {",out,"}"
        return out
    finally:
        lock.release()

    

if __name__ == '__main__':
    from wsgiref.simple_server import make_server
#     logging.basicConfig(level=logging.DEBUG)
#     logging.getLogger('spyne.protocol.xml').setLevel(logging.DEBUG)
    application = Application([DiminishedTriadService],
        tns='DiminishedTriadService',
        in_protocol=HttpRpc(validator='soft'),
        out_protocol=JsonDocument()
    )
    wsgi_app = WsgiApplication(application)
    server = make_server('127.0.0.1', 50001, wsgi_app)
    server.serve_forever()
    
    
    

getAOF returning  *2
$6
SELECT
$1
0
*3
$3
set
$3
foo
$3
bar
*3
$3
set
$3
goo
$3
dar



127.0.0.1 - - [24/May/2015 20:51:53] "GET /getAOFLog HTTP/1.1" 200 125


KeyboardInterrupt: 